<a href="https://colab.research.google.com/github/kimanirobbi/wk-3-ai/blob/main/bonus_task_flask_deployment_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This script outlines the structure for the Bonus Task: Deploying the MNIST model
# using Flask. To run this, you would need to save the model from Task 2 first.
#
# Prerequisites:
# 1. Install Flask and TensorFlow: pip install flask tensorflow numpy
# 2. Save the trained model from task2_deep_learning_cnn.py:
#    model.save('mnist_cnn_model.h5')
# 3. Create a 'static' folder for CSS/JS (optional) and a 'templates' folder for index.html

import os
import io
import base64
import numpy as np
from PIL import Image
from flask import Flask, render_template, request, jsonify
import tensorflow as tf
from tensorflow.keras.models import load_model

# Configuration for Flask and Model Loading
app = Flask(__name__)
# The model path should be updated once the model is trained and saved
MODEL_PATH = 'mnist_cnn_model.h5'

# Load the trained model globally when the app starts
try:
    # Suppress TensorFlow logging to keep console clean
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    # Load the model structure and weights
    MODEL = load_model(MODEL_PATH)
    MODEL.predict(np.zeros((1, 28, 28, 1))) # Warm up the model
    print(f"Successfully loaded model from: {MODEL_PATH}")
except Exception as e:
    # If the model file is not found (common when running this file alone)
    MODEL = None
    print(f"Warning: Could not load the model file ({MODEL_PATH}).")
    print("To run this, ensure the model is saved correctly after Task 2.")

# --------------------------------------------------------------------
# Helper Functions
# --------------------------------------------------------------------

def preprocess_image(img_data_url):
    """Converts a base64 Data URL (from a canvas) into a preprocessed numpy array."""

    # The image data is split to get the base64 part
    encoded_data = img_data_url.split(',')[1]
    binary_data = base64.b64decode(encoded_data)

    # Open the image using PIL
    img = Image.open(io.BytesIO(binary_data))

    # The image from canvas is usually 280x280 (or similar) and RGBA.
    # 1. Resize to 28x28 (required by MNIST model)
    # 2. Convert to grayscale ('L' mode)
    # 3. Invert colors (MNIST is black text on white background, canvas is usually reverse)
    img = img.resize((28, 28)).convert('L')

    # Convert image to numpy array, normalize, and invert colors (255 - pixel_value)
    img_array = np.array(img).astype('float32')
    img_array = 255.0 - img_array # Invert for black background

    # Normalize to 0-1 range and reshape for CNN (1, 28, 28, 1)
    img_array /= 255.0
    img_array = img_array.reshape(1, 28, 28, 1)

    return img_array

# --------------------------------------------------------------------
# Flask Routes
# --------------------------------------------------------------------

@app.route('/')
def index():
    """Renders the main page with the drawing canvas."""
    # In a real deployment, you would need a file named 'index.html' in a 'templates' folder.
    # For this deliverable, we provide the full HTML/JS in a single function return (simulated)
    return render_template('index_simulated.html')


@app.route('/predict', methods=['POST'])
def predict():
    """Handles the prediction request from the web interface."""
    if MODEL is None:
        return jsonify({'error': 'Model not loaded.'}), 500

    data = request.get_json()
    img_data_url = data.get('image')

    if not img_data_url:
        return jsonify({'error': 'No image data provided.'}), 400

    try:
        # Preprocess the image
        processed_image = preprocess_image(img_data_url)

        # Make the prediction
        predictions = MODEL.predict(processed_image)
        predicted_class = int(np.argmax(predictions[0]))
        confidence = float(np.max(predictions[0]))

        # Return the result
        return jsonify({
            'success': True,
            'prediction': predicted_class,
            'confidence': f"{confidence*100:.2f}%"
        })

    except Exception as e:
        print(f"Prediction Error: {e}")
        return jsonify({'success': False, 'error': str(e)}), 500

# --------------------------------------------------------------------
# Simulated HTML Template (for demonstration of the deployment concept)
# --------------------------------------------------------------------

# Since I cannot generate a separate index.html, I will provide a conceptual
# explanation of the front-end needed for the deployment.

# Conceptual Front-End (index.html):
# 1. <canvas> element for drawing.
# 2. JavaScript to handle mouse/touch events to draw on the canvas.
# 3. A "Predict" button that converts the canvas content to a base64 Data URL.
# 4. An AJAX call (fetch/XMLHttpRequest) to send the Data URL to the /predict endpoint.
# 5. A display area to show the prediction and confidence score.

if __name__ == '__main__':
    print("\n--- Bonus Task: Flask Deployment Setup ---")
    print("This script is a conceptual framework.")
    print(f"To run: Save model as '{MODEL_PATH}' and then execute the script.")
    print("Access the app at http://127.0.0.1:5000/")
    # app.run(debug=True) # Uncomment to run the Flask app locally

To run this, ensure the model is saved correctly after Task 2.

--- Bonus Task: Flask Deployment Setup ---
This script is a conceptual framework.
To run: Save model as 'mnist_cnn_model.h5' and then execute the script.
Access the app at http://127.0.0.1:5000/


In [ ]:
%pip install tensorflow

In [ ]:
!python bonus_task_flask_deployment.py

python3: can't open file '/content/bonus_task_flask_deployment.py': [Errno 2] No such file or directory


In [ ]:
import os
import io
import base64
import numpy as np
from PIL import Image
from flask import Flask, render_template, request, jsonify
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- Flask Configuration ---
app = Flask(__name__)

# FIX: Use os.path methods to ensure the model path is absolute and correct.
# This calculates the path relative to where app.py is located.
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
MODEL_PATH = os.path.join(BASE_DIR, 'mnist_cnn_model.h5')

# Load the trained model globally when the app starts.
try:
    # Suppress TensorFlow logging to keep console clean
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    # Load the model structure and weights using the fixed path
    MODEL = load_model(MODEL_PATH)
    # Warm up the model for the first prediction
    MODEL.predict(np.zeros((1, 28, 28, 1)))
    print(f"[INFO] Successfully loaded model from: {MODEL_PATH}")
except Exception as e:
    MODEL = None
    print(f"[ERROR] Could not load the model file ({MODEL_PATH}).")
    print(f"Please ensure the file exists in the same folder as app.py. Error: {e}")

# --- Helper Function for Image Preprocessing (Remains the same) ---
def preprocess_image(img_data_url):
    """Converts a base64 Data URL (from a canvas) into a preprocessed numpy array."""

    encoded_data = img_data_url.split(',')[1]
    binary_data = base64.b64decode(encoded_data)
    img = Image.open(io.BytesIO(binary_data))

    img = img.resize((28, 28)).convert('L')

    img_array = np.array(img).astype('float32')
    img_array = 255.0 - img_array # Invert for black background

    img_array /= 255.0
    img_array = np.clip(img_array, 0, 1)

    img_array = img_array.reshape(1, 28, 28, 1)

    return img_array

# --- Flask Routes (Remain the same) ---

@app.route('/')
def index():
    """Renders the main template containing the drawing interface."""
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    """API endpoint to receive canvas data and return the prediction."""
    if MODEL is None:
        return jsonify({'error': 'Model not loaded on server.'}), 500

    data = request.get_json()
    img_data_url = data.get('image')

    if not img_data_url:
        return jsonify({'error': 'No image data provided.'}), 400

    try:
        processed_image = preprocess_image(img_data_url)
        predictions = MODEL.predict(processed_image)
        predicted_class = int(np.argmax(predictions[0]))
        confidence = float(np.max(predictions[0]))

        return jsonify({
            'success': True,
            'prediction': predicted_class,
            'confidence': f"{confidence*100:.2f}%"
        })

    except Exception as e:
        print(f"Prediction Error: {e}")
        return jsonify({'success': False, 'error': 'Internal server error during prediction.'}), 500

if __name__ == '__main__':
    print("\n--- MNIST CNN Deployment Ready ---")
    print(f"Attempting to load model from: {MODEL_PATH}")
    print("If this fails, please ensure 'mnist_cnn_model.h5' is next to app.py.")
    app.run(debug=True)
